## 作業

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量


In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
def cifar_generator(image_array, batch_size=32):
    while True:
        for indexs in range(0, len(image_array), batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels
            
            


In [0]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_function = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [24]:
model = Sequential()
model.add(Conv2D(16, (5, 5), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 32, 32, 16)        1216      
_________________________________________________________________
activation_32 (Activation)   (None, 32, 32, 16)        0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 32)        12832     
_________________________________________________________________
activation_33 (Activation)   (None, 12, 12, 32)        0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 2, 2, 128)         102528    
__________

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])



history = model.fit_generator(cifar_generator(x_train),
                              epochs=50,
                              steps_per_epoch=len(x_train)/32,
                              verbose=1,
                              validation_data=(x_test, y_test),
                              callbacks = [learning_rate_function])

'''
  train_history = model.fit_generator(train_generator, 
                                    steps_per_epoch=300, 
                                    epochs=30, 
                                    verbose=1,
                                    validation_data=(x_val, y_val_onehot),
                                    callbacks=[learning_rate_function])
'''
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
1563/1562 [==============================] - 10s 6ms/step - loss: 1.6472 - acc: 0.4034 - val_loss: 1.6553 - val_acc: 0.4448
Epoch 2/50
1563/1562 [==============================] - 9s 6ms/step - loss: 1.2732 - acc: 0.5551 - val_loss: 1.2956 - val_acc: 0.5466
Epoch 3/50
1563/1562 [==============================] - 9s 6ms/step - loss: 1.1274 - acc: 0.6131 - val_loss: 1.2334 - val_acc: 0.5874
Epoch 4/50
1563/1562 [==============================] - 9s 6ms/step - loss: 1.0522 - acc: 0.6433 - val_loss: 1.3258 - val_acc: 0.5699
Epoch 5/50
1563/1562 [==============================] - 9s 6ms/step - loss: 1.0232 - acc: 0.6614 - val_loss: 1.2468 - val_acc: 0.6317
Epoch 6/50
1563/1562 [==============================] - 9s 6ms/step - loss: 1.0343 - acc: 0.6597 - val_loss: 1.1659 - val_acc: 0.6211
Epoch 7/50
1563/1562 [==============================] - 9s 6ms/step - loss: 1.0694 - acc: 0.6516 - val_loss: 1.3027 - val_acc: 0.6191
Epoch 8/50
1563/1562 [==============================] - 9s 6m